In [2]:
pip install openpyxl


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: C:\Users\Shreya Kolte\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

# Load the Excel file
file_path = "C:/Users/Shreya Kolte/OneDrive/Desktop/global_econ_intelligence_project/Data/Raw/country.xlsx"

df_raw = pd.read_excel(file_path)

# Preview the raw data
print("🔍 Raw shape:", df_raw.shape)
print(df_raw.head())


🔍 Raw shape: (8972, 29)
   year  CTY_CODE         CTYNAME     IJAN     IFEB     IMAR     IAPR  \
0  1997         3  European Union  11644.9  11764.7  13616.9  13577.4   
1  1998         3  European Union  12883.4  12954.5  15970.6  15144.0   
2  1999         3  European Union  13700.7  14788.6  17131.3  16129.8   
3  2000         3  European Union  15771.3  17227.4  20757.7  18285.4   
4  2001         3  European Union  19413.6  18491.1  20720.2  19677.4   

      IMAY     IJUN     IJUL  ...     EAPR     EMAY     EJUN     EJUL  \
0  13528.9  13183.1  14865.8  ...  12450.5  12270.2  11868.4  10657.2   
1  14374.4  15502.9  16224.6  ...  12271.3  12951.0  12666.8  11275.9   
2  16035.2  17217.8  17873.4  ...  13091.2  12931.7  12393.7  11484.5   
3  19341.8  18583.1  18982.7  ...  13860.9  14224.1  14220.4  12235.6   
4  19596.3  18334.9  20129.0  ...  13949.0  14427.2  13996.9  11531.9   

      EAUG     ESEP     EOCT     ENOV     EDEC       EYR  
0  11042.0  11833.8  12773.9  11994.2  

In [4]:
# Filter only 2024 data
df_2024 = df_raw[df_raw["year"] == 2024].copy()

# Keep only import columns
import_cols = ['CTYNAME', 'IJAN', 'IFEB', 'IMAR', 'IAPR', 'IMAY', 'IJUN',
               'IJUL', 'IAUG', 'ISEP', 'IOCT', 'INOV', 'IDEC']
df_imports = df_2024[import_cols]

# Rename months
month_map = {
    'IJAN': '2024-01', 'IFEB': '2024-02', 'IMAR': '2024-03', 'IAPR': '2024-04',
    'IMAY': '2024-05', 'IJUN': '2024-06', 'IJUL': '2024-07', 'IAUG': '2024-08',
    'ISEP': '2024-09', 'IOCT': '2024-10', 'INOV': '2024-11', 'IDEC': '2024-12'
}
df_imports.rename(columns=month_map, inplace=True)

# Reshape to long format
df_long = df_imports.melt(id_vars='CTYNAME', var_name='Month', value_name='Import_Value')
df_long['Month'] = pd.to_datetime(df_long['Month'])
df_long.rename(columns={'CTYNAME': 'Country'}, inplace=True)

# View tidy version
df_long.head()

,Country,Month,Import_Value
0,European Union,2024-01-01,46556.315322
1,"World, Seasonally Adjusted",2024-01-01,259364.777642
2,Advanced Technology Products,2024-01-01,53188.254647
3,South and Central America,2024-01-01,13064.909264
4,North America,2024-01-01,71351.448154


In [5]:
# Step 1: Filter only 2024 data
df_2024 = df_raw[df_raw["year"] == 2024].copy()

# Step 2: Select export columns
export_cols = ['CTYNAME', 'EJAN', 'EFEB', 'EMAR', 'EAPR', 'EMAY', 'EJUN',
               'EJUL', 'EAUG', 'ESEP', 'EOCT', 'ENOV', 'EDEC']
df_exports = df_2024[export_cols]

# Step 3: Rename month columns
month_map_exports = {
    'EJAN': '2024-01', 'EFEB': '2024-02', 'EMAR': '2024-03', 'EAPR': '2024-04',
    'EMAY': '2024-05', 'EJUN': '2024-06', 'EJUL': '2024-07', 'EAUG': '2024-08',
    'ESEP': '2024-09', 'EOCT': '2024-10', 'ENOV': '2024-11', 'EDEC': '2024-12'
}
df_exports.rename(columns=month_map_exports, inplace=True)

# Step 4: Reshape to long format
df_exports_long = df_exports.melt(id_vars='CTYNAME', var_name='Month', value_name='Export_Value')
df_exports_long['Month'] = pd.to_datetime(df_exports_long['Month'])
df_exports_long.rename(columns={'CTYNAME': 'Country'}, inplace=True)

# Final export tidy data
df_exports_long.head()


,Country,Month,Export_Value
0,European Union,2024-01-01,29808.028390
1,"World, Seasonally Adjusted",2024-01-01,169081.115983
2,Advanced Technology Products,2024-01-01,33839.003791
3,South and Central America,2024-01-01,15816.868643
4,North America,2024-01-01,52784.854643


In [6]:
# Merge imports and exports on Country + Month
df_merged = pd.merge(
    df_long,                 # Imports
    df_exports_long,         # Exports
    on=['Country', 'Month'],
    how='outer'              # In case some values are missing
)

# Calculate Trade Balance
df_merged['Trade_Balance'] = df_merged['Export_Value'] - df_merged['Import_Value']

# Preview
df_merged.sort_values(by=['Country', 'Month'], inplace=True)
df_merged.head()


,Country,Month,Import_Value,Export_Value,Trade_Balance
0,Advanced Technology Products,2024-01-01,53188.254647,33839.003791,-19349.250856
1,Advanced Technology Products,2024-02-01,50239.529505,33269.672344,-16969.857161
2,Advanced Technology Products,2024-03-01,57260.563237,39024.418635,-18236.144602
3,Advanced Technology Products,2024-04-01,61381.770734,36165.674552,-25216.096182
4,Advanced Technology Products,2024-05-01,61900.930853,37337.173882,-24563.756971


In [7]:
# Export the merged & cleaned trade data to CSV
df_merged.to_csv("C:/Users/Shreya Kolte/OneDrive/Desktop/global_econ_intelligence_project/Data/Clean/us_trade_balance_2024.csv", index=False)
print("Clean trade data exported to Data/Clean/")

Clean trade data exported to Data/Clean/
